In [14]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re
import json

# Selenium (works perfectly)

In [15]:
# Define Browser Options (hide the actual browser)
chrome_options = Options()
chrome_options.add_argument("--headless") # Hides the browser window

In [16]:
# a list with all possible urls i.e. org_url start with some random organisation
FILE_PATH_FOLDER = 'scrape/'
org_url = ['https://stiftungen.stiftungschweiz.ch/organizations/annette-ten-zeldam-stiftung']
driver = webdriver.Chrome(executable_path='Chrome_Webdriver/chromedriver', options=chrome_options)
org_details = []
org_data = pd.DataFrame(columns=['foundation','website','phone','adress_street','address_house_number','address_postal_code', 'address_city', 'scope',
                                'topics', 'targets', ])

In [17]:
# add all urls to org_url list (this only seems possible when using an actual browser)
# use regex to extract data from HTML
# paste all data into org_data
error_url = []
for url in org_url:
    try:
        driver.get(url)
        time.sleep(2.1)
        print(url)
        html = driver.page_source
        org = re.findall(re.compile(r"(?<= organization: )(.*)(?=,.*jobs:)", re.DOTALL), html)
        dict_org = json.loads(org[0])
        topics = re.findall(re.compile(r"(?<=nksList: )(.*)(?=,.*nksParentNames:)", re.DOTALL), html)
        dict_topics = json.loads(topics[0])
        
        topics = []
        for key, value in dict_topics['2'].items():
            for topic in value:
                topics.append(topic['name'])

        people = []        
        try:
            people = re.findall(re.compile(r"(?<=people: )(.*)(?=,.*projects: \[\],)", re.DOTALL), html)
            people = json.loads(people[0])
            dict_people = {}
            for dict_pers in people:
                try:
                    dict_people[dict_pers['organization_function']['de']] = dict_pers['fulldata']
                except:
                    continue
        except:
            continue

        targets = []
        try:
            targets = [target['name'] for target in dict_topics['3'][0]]
        except:
            continue

        org_data = org_data.append({
            'foundation': dict_org['title'],
            'website' : dict_org['site'],
            'phone' : dict_org['phone'],
            'adress_street' : dict_org['address_street'],
            'address_house_number' : dict_org['address_house_number'],
            'address_postal_code' : dict_org['address_postal_code'], 
            'address_city' : dict_org['address_city'], 
            'scope' : [area['location'] for area in dict_org['areas']],
                'topics' : topics,
                'targets': targets,
                'people' : dict_people
                            }, 
                            ignore_index = True)

        for element in driver.find_elements_by_xpath("//a[@class='proj-item']"):
            if element.get_attribute('href') not in org_url:
                org_url.append(element.get_attribute('href'))
        
    except:
        error_url.append(url)
        print('ERROR___url: ' + url)
        continue
            

organizations/stiftung-pro-sport-sargans
https://stiftungen.stiftungschweiz.ch/organizations/stiftung-vinetum
https://stiftungen.stiftungschweiz.ch/organizations/schweizerisch-liechtensteinische-stiftung-fuer-archaeologische-forschungen-im-ausland
https://stiftungen.stiftungschweiz.ch/organizations/volkshausstiftung-zuerich
https://stiftungen.stiftungschweiz.ch/organizations/stiftung-cvjm-haus-zum-kornfeld
https://stiftungen.stiftungschweiz.ch/organizations/cevi-wartburg
https://stiftungen.stiftungschweiz.ch/organizations/ortsmuseum-thalwil
https://stiftungen.stiftungschweiz.ch/organizations/stiftung-ortsmuseum-urdorf
https://stiftungen.stiftungschweiz.ch/organizations/stiftung-hochdorf-fuer-residio
https://stiftungen.stiftungschweiz.ch/organizations/ernst-kohler-stiftung
https://stiftungen.stiftungschweiz.ch/organizations/stiftung-sternwarte-oberberg-schuepfheim
https://stiftungen.stiftungschweiz.ch/organizations/stiftung-telephonica-im-greuterhof-islikon-tg
https://stiftungen.stiftun

In [18]:
error_url

['https://stiftungen.stiftungschweiz.ch/organizations/verein-step-up-school',
 'https://stiftungen.stiftungschweiz.ch/organizations/fondazione-culture-e-musei',
 'https://stiftungen.stiftungschweiz.ch/organizations/fondation-du-reseau-sante-mentale',
 'https://stiftungen.stiftungschweiz.ch/organizations/fondation-generosite',
 'https://stiftungen.stiftungschweiz.ch/organizations/arche-im-nauen',
 'https://stiftungen.stiftungschweiz.ch/organizations/tierschutz-beider-basel',
 'https://stiftungen.stiftungschweiz.ch/organizations/ecosolidar',
 'https://stiftungen.stiftungschweiz.ch/organizations/pfarrpfrundstiftung-gersau',
 'https://stiftungen.stiftungschweiz.ch/organizations/pfarrpfrundstiftung-riemenstalden',
 'https://stiftungen.stiftungschweiz.ch/organizations/herzensbilder-ch',
 'https://stiftungen.stiftungschweiz.ch/organizations/pfarrkirchenstiftung-gersau',
 'https://stiftungen.stiftungschweiz.ch/organizations/kapellstiftung-maria-hilf-kindli',
 'https://stiftungen.stiftungschwei

In [1]:
len(org_url)

NameError: name 'org_url' is not defined

In [ ]:
len(org_url) - len(org_data)

In [12]:
org_data

,foundation,website,phone,adress_street,address_house_number,address_postal_code,address_city,scope,topics,targets,people
0,Annette ten Zeldam Stiftung,None,None,Hirschgässlein,30,4051,Basel,[Schweiz],"[Soziales allgemein, Entwicklungszusammenarbei...","[Offen/alle, Kinder, Menschen mit körperlicher...","{'Mitglied des Stiftungsrates': 'Braun, Annett..."
1,Ernesto Bertarelli Foundation,None,None,Suterstrasse,4,3780,Gstaad,[Schweiz],"[Breitensport, Umwelt- und Naturschutz, Oekolo...","[Offen/alle, Kinder, Menschen mit körperlicher...","{'Präsident des Stiftungsrates': 'Bertarelli, ..."
2,Annette Ringier-Stiftung,None,None,Spielhof,14a,8750,Glarus,[Schweiz],"[Kunst und Kultur allgemein, Soziales allgemei...","[Offen/alle, Arme, Bedürftige, Verschiedene De...","{'Mitglied': 'Fuchs, Erwin, von Appenzell und ..."
3,Clara-Stiftung,None,None,Querstrasse,5,8212,Neuhausen am Rheinfall,[Schweiz],"[Kunst und Kultur allgemein, Gesellschaftspoli...","[Offen/alle, Kinder, Jugendliche, Frauen, Mütter]",{'Präsident des Stiftungsrates': 'Hackenschmid...
4,DROSOS STIFTUNG,www.drosos.org/,+41 43 300 54 00,Obstgartenstrasse,19,8006,Zürich,[Schweiz],"[Kunst und Kultur allgemein, Soziales allgemei...","[Offen/alle, Menschen mit körperlicher und/ode...","{None: 'Aebischer, Jean-Claude, von Schmitten ..."
...,...,...,...,...,...,...,...,...,...,...,...
4775,Fondation pour la recherche de la cardiologie ...,None,None,Rue Micheli-du-Crest,24,1205,Genève,"[Schweiz, Geneva, Switzerland]",[Medizinische Forschung Herz-Kreislauf-Erkrank...,[Offen/alle],"{'Mitglied': 'Polikar Ralf, de Lausanne, à Pra..."
4776,Fondation pour la Promotion de l'Unité et du T...,None,None,Rue du Rhône,30,1204,Genève,[Schweiz],[Christlicher Glaube],[Offen/alle],"{'Mitglied, Präsident': 'Vischer Lukas, de Bâl..."
4777,GULF RESEARCH CENTER FOUNDATION,None,None,Avenue de France,23,1202,Genève,"[Schweiz, Bahrain, Kuwait, Oman, Qatar, United...",[Soziale und gesellschaftspolitische Forschung...,"[Offen/alle, Berufsgruppe entsprechend der inh...","{'Mitglied, Präsident': 'Sager Abdulaziz Othma..."
4778,Fondation Islamique d'Ahl-el-Beit (S) de Suiss...,None,+41 22 340 00 59,Avenue du Lignon,48,1219,Le Lignon,[Schweiz],"[Kunst und Kultur allgemein, Anderer Glaube (z...","[Offen/alle, Religiöse Gemeinschaften]","{'Mitglied, Präsident': 'Moucheimech Hadi, de ..."


In [21]:
org_data.to_csv('Foundations_CH.csv', index = False)

# Trial and Error with Beautiful Soup etc.

In [46]:
 for each_org in org_list:
    # Getting link:
    org_link = each_org.find_elements_by_xpath("//href")
    print(each_org)
    #job_info = [job_title.text, job_company.text, job_location.text, job_summary.text, job_publish_date.text]
    # Saving into job_details
    org_details.append(org_link)
driver.quit()

<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="f71e6792-8092-4293-a1cd-35930c7005b7")>
<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="b9830347-bf10-41aa-98e5-e9bdf9a316e7")>
<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="3e8a8d79-78e4-40ae-9dc9-e1137e4d62e5")>
<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="dd4a52a3-8876-453d-8325-ea3873c26bd7")>
<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="d8372cd4-983e-47c2-b6b2-f09c86b057ab")>
<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="9083f758-9793-4751-927a-19896c75005b")>
<selenium.webdriver.remote.webelement.WebElement (session="2e2b1548572af4fe4003940c6dddf5cb", element="a86608b2-0ab9-4f1a-b4d5-6846fbf0d19d")>

In [32]:
url = 'https://stiftungen.stiftungschweiz.ch/organizations/annette-ten-zeldam-stiftung'


r = requests.get(url)

doc = BeautifulSoup(r.text, "html.parser")

foundation = doc.select_one(".d-none h2 span").text
info = doc.select_one(".d-none h2 span").text
purpose = doc.select_one(".d-none p").text
link = doc.select(".slick-active a")

#topics = doc.find_all('script')
print('foundation: ' + foundation + 
                            ' | info: ' + info +
                            ' | purpose: ' + purpose)

link

foundation: Annette ten Zeldam Stiftung | info: Annette ten Zeldam Stiftung | purpose:  Die Stiftung bezweckt die finanzielle Unterstützung von Institutionen und Organisationen gemeinnütziger oder wohltätiger Art, welche sich insbesondere um sozial, körperlich oder seelisch benachteiligte Kinder sowie um den weltweiten Fairnessgedanken im Arbeits- und Sozialbereich kümmern. Der Wirkungskreis der Stiftung erstreckt sich über die Schweiz hinaus. Die Stiftung hat gemeinnützigen Charakter und verfolgt keinerlei Erwerbszweck.


[]

In [67]:
org = re.findall(re.compile(r"(?<= organization: )(.*)(?=,.*jobs:)", re.DOTALL), r.text)
org

['{"id":"19252","slug":"annette-ten-zeldam-stiftung","slug_id":"19252-annette-ten-zeldam-stiftung","title":"Annette ten Zeldam Stiftung","effective_type":1,"legal_type":1,"site":null,"phone":null,"email":null,"address_street":"Hirschg\\u00e4sslein","address_house_number":"30","address_postal_code":4051,"address_city":"Basel","address_line_2":"c\\/o NEOVIUS AG","address_country":"CH","address_note":null,"show_address_note":0,"custom_address_street":null,"custom_address_house_number":null,"custom_address_postal_code":null,"custom_address_city":null,"custom_address_line_2":null,"custom_address_country":null,"summary":null,"strategy":null,"goal":null,"purpose_zefix":"Die Stiftung bezweckt die finanzielle Unterst\\u00fctzung von Institutionen und Organisationen gemeinn\\u00fctziger oder wohlt\\u00e4tiger Art, welche sich insbesondere um sozial, k\\u00f6rperlich oder seelisch benachteiligte Kinder sowie um den weltweiten Fairnessgedanken im Arbeits- und Sozialbereich k\\u00fcmmern. Der Wirku

In [68]:
dict_org = json.loads(org[0])
dict_org

{'id': '19252',
 'slug': 'annette-ten-zeldam-stiftung',
 'slug_id': '19252-annette-ten-zeldam-stiftung',
 'title': 'Annette ten Zeldam Stiftung',
 'effective_type': 1,
 'legal_type': 1,
 'site': None,
 'phone': None,
 'email': None,
 'address_street': 'Hirschgässlein',
 'address_house_number': '30',
 'address_postal_code': 4051,
 'address_city': 'Basel',
 'address_line_2': 'c/o NEOVIUS AG',
 'address_country': 'CH',
 'address_note': None,
 'show_address_note': 0,
 'custom_address_street': None,
 'custom_address_house_number': None,
 'custom_address_postal_code': None,
 'custom_address_city': None,
 'custom_address_line_2': None,
 'custom_address_country': None,
 'summary': None,
 'strategy': None,
 'goal': None,
 'purpose_zefix': 'Die Stiftung bezweckt die finanzielle Unterstützung von Institutionen und Organisationen gemeinnütziger oder wohltätiger Art, welche sich insbesondere um sozial, körperlich oder seelisch benachteiligte Kinder sowie um den weltweiten Fairnessgedanken im Arbeit

In [76]:
dict_org['title']
dict_org['site']
dict_org['phone']
dict_org['address_street']
dict_org['address_house_number']
dict_org['address_postal_code']
dict_org['address_city']

'Basel'

In [70]:
for area in dict_org['areas']:
    print(area['location'])

Schweiz


In [71]:
topics = re.findall(re.compile(r"(?<=nksList: )(.*)(?=,.*nksParentNames:)", re.DOTALL), r.text)
dict_topics = json.loads(topics[0])
dict_topics

{'2': {'115': [{'id': 22,
    'name': 'Soziales allgemein',
    'name_en': 'Social general',
    'name_de': 'Soziales allgemein',
    'name_fr': 'Allgemein',
    'kennziffer': '0301',
    'nks_systemelemente_id': 2,
    'order_by': 99999,
    'parent_id': 115,
    'parent_kennziffer': '0300',
    'parent_name_en': 'Soziales (karitative Zwecke, Unterstützung Bedürftige)',
    'parent_name_de': 'Soziales (karitative Zwecke, Unterstützung Bedürftige)',
    'parent_name_fr': 'Soziales (karitative Zwecke, Unterstützung Bedürftige)',
    'nks_systemelemente': {'id': 2,
     'name': 'Themenbereich',
     'name_en': 'Effect areas (with characteristics)',
     'name_de': 'Themenbereich',
     'name_fr': "Domaines d'action (avec variantes)",
     'kennziffer': 'B'}}],
  '110': [{'id': 57,
    'name': 'Entwicklungszusammenarbeit allgemein',
    'name_en': 'Entwicklungszusammenarbeit allgemein',
    'name_de': 'Entwicklungszusammenarbeit allgemein',
    'name_fr': 'Entwicklungszusammenarbeit allge

In [72]:
for key, value in dict_topics['2'].items():
    for topic in value:
        print(topic['name'])

Soziales allgemein
Entwicklungszusammenarbeit allgemein
Politik, Gesellschaft, Wirtschaft verschiedenes 


In [73]:
for target in dict_topics['3'][0]:
    print(target['name'])

Offen/alle
Kinder
Menschen mit körperlicher und/oder geistiger Beeinträchtigung
Kranke (z.B. psychisch und physisch akut Kranke, Suchtkranke)


In [74]:
people = re.findall(re.compile(r"(?<=people: )(.*)(?=,.*projects: \[\],)", re.DOTALL), r.text)
people = json.loads(people[0])
people

[{'id': 181706,
  'merged_into': None,
  'a_organization_id': 0,
  'first_name': 'Peter',
  'last_name': 'Göggel',
  'full_name': 'Peter Göggel',
  'email': None,
  'phone': None,
  'mobile_phone': None,
  'web': None,
  'country': 'Switzerland',
  'location': 'Oberwil',
  'canton': 'Basel-Landschaft',
  'gender': None,
  'postal_code': None,
  'address': None,
  'street_nr': None,
  'company_name': None,
  'title': '',
  'function': 'member of the Foundation Board',
  'localized_function': {'origin': 'member of the Foundation Board',
   'en': None,
   'de': 'Mitglied des Stiftungsrates',
   'fr': None,
   'it': None},
  'organization_function': {'origin': 'member of the Foundation Board',
   'en': None,
   'de': 'Mitglied des Stiftungsrates',
   'fr': None,
   'it': None},
  'from': ' von Luzern',
  'in': ' in Oberwil (BL)',
  'fulldata': 'Göggel, Peter, von Luzern, in Oberwil (BL)',
  'check': False,
  'private': None,
  'datasource_id': 'SHAB',
  'zefix_id': 122238,
  'auditor': Non

In [75]:
for dict_pers in people:
    print(dict_pers['first_name'])
    print(dict_pers['last_name'])
    print(dict_pers['organization_function']['de'])
    print(dict_pers['fulldata'])

Peter
Göggel
Mitglied des Stiftungsrates
Göggel, Peter, von Luzern, in Oberwil (BL)
Annette
Braun
Mitglied des Stiftungsrates
Braun, Annette, von Basel, in Dornach
None
None
Auditor
BGS Wirtschaftsprüfungs AG (CHE-205.618.556), in Zürich
Rolf
Hamburger
Präsident des Stiftungsrates
Hamburger, Rolf, von Gaiserwald, in St. Gallen
